In [1]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img
from PIL import Image

2024-09-02 09:59:04.413636: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-02 09:59:04.423896: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-02 09:59:04.504461: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-02 09:59:04.587218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-02 09:59:04.653537: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
#paths to files
image_dir = "../raw_data/facial_segmentation/image/"
mask_dir = "../raw_data/facial_segmentation/masks/seg/"

#variables that hold file names
image_filenames = os.listdir(image_dir)
mask_filenames = os.listdir(mask_dir)

#sorting file names
image_filenames.sort()
mask_filenames.sort()

In [3]:
size_of_file = 5000
# training on size of file number of images training
image_filenames = image_filenames[:size_of_file]
mask_filenames = mask_filenames[:size_of_file]

IMG_SIZE = (256, 256)

images = []
masks = []

In [4]:
# Helper function to check if a path is a file
def is_file(path):
    return os.path.isfile(path)

# Loop through each pair of image and mask filenames
for image_filename, mask_filename in zip(image_filenames, mask_filenames):
    # Construct full file paths
    image_path = os.path.join(image_dir, image_filename)
    mask_path = os.path.join(mask_dir, mask_filename)

    # Check if files exist and are files
    if not is_file(image_path):
        continue
    if not is_file(mask_path):
        continue

    # Load and preprocess the face image
    img = load_img(image_path, target_size=IMG_SIZE)
    img = img_to_array(img) / 255.0  # Normalize to [0, 1]

    # Load and preprocess the mask
    mask = load_img(mask_path, target_size=IMG_SIZE, color_mode="grayscale")
    mask = img_to_array(mask)  # Normalize to [0, 1]

    # Append the processed image and mask to the respective lists
    images.append(img)
    masks.append(mask)

# Convert lists to numpy arrays
images = np.array(images)
masks = np.array(masks)

In [5]:
# Split into training and validation sn, y_test = train_test_split(images, masksets (80% training, 20% validation)
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.2, random_state=42)
# Print shapes to confirm the split
print(f"Training data shape: {X_train.shape}, Training masks shape: {y_train.shape}")
print(f"Test data shape: {X_test.shape}, Test masks shape: {y_test.shape}")

Training data shape: (4000, 256, 256, 3), Training masks shape: (4000, 256, 256, 1)
Test data shape: (1000, 256, 256, 3), Test masks shape: (1000, 256, 256, 1)


In [6]:
# Define valid values to keep
valid_values = {1, 2, 3, 4, 5, 7, 9, 10}

y_train = np.where(np.isin(y_train, list(valid_values)), y_train, 0)

In [7]:
label_mapping = {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 7: 6, 9: 7, 10: 8}

# Apply the mapping
for old_value, new_value in label_mapping.items():
    y_train[y_train == old_value] = new_value

: 

In [8]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train,num_classes = 9)

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape, num_classes):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(num_classes, 1, padding="same", activation="softmax")(d4)

    model = Model(inputs, outputs)
    return model


if __name__ == "__main__":
    input_shape = (256, 256, 3)
    model = build_unet(input_shape, 9)
    model.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, UpSampling2D, Concatenate, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint = ModelCheckpoint('unet_model2.h5', save_best_only=True)
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=50, batch_size = 16, callbacks=[checkpoint, early_stopping], validation_split = 0.2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

predicted_mask = model.predict(np.expand_dims(X_train[0], axis=0))

print(predicted_mask.shape)
# Assuming one-hot encoded mask with 9 classes
predicted_mask = np.argmax(predicted_mask, axis=-1)  # Select channel with highest probability
predicted_mask = np.reshape(predicted_mask, (256, 256))
plt.imshow(predicted_mask, cmap='gray')


In [ ]:
plt.imshow(X_train[0])
